In [8]:
# tui.py

def display_menu():
    selected_option = input("""Please select one of the following options:
[1] Retrieve Demographic Information by patient ID
[2] Retrieve Medical History Details by ethnicity
[3] Retrieve Treatment Details
[4] Retrieve the frequency of data occurrences by column
[5] Identify Top 3 Treatments
[6] Analyse Average White Blood Cell Counts
[7] Smoking Packs Analysis by Tumor Location
[10] Exit application

    """)
    return selected_option.strip()

def display_A1(found, patient_info):
    if found == True:
        print("Age: ", patient_info["Age"], "\nGender: ", patient_info["Gender"],
              "\nSmoking history: ", patient_info["Smoking history"], "\nEthnicity: ", patient_info["Ethnicity"], "\n")
    else:
        # Else if - user input cannot be found
        print("Patient ID could not be found")

def display_A2(ethnicity_history, count):
    for i in range(0, count, 4):
        print("\nFamily history of lung cancer: ", ethnicity_history[i],
              "\nComorbidities of diabetes disease: ", ethnicity_history[i+1],
              "\nComorbidities of kidney disease: ", ethnicity_history[i+2],
              "\nHemoglobin level associated with ethnicity: ", ethnicity_history[i+3], "\n")

def display_A3(found, treatment_details, count):
    if found == True:
        for i in range(0, count, 4):
            print("\nAge: ", treatment_details[i],
                  "\nTumor size: ", treatment_details[i+1],
                  "\nTumor location: ", treatment_details[i+2],
                  "\nTumor stage: ", treatment_details[i+3], "\n")
    else:
        # Else if - user input cannot be found
        print("Treatment type could not be found")

def display_A4(found, item_dict):
    if found == True:

        sorted_item_dict = dict(sorted(item_dict.items()))

        for k, v in sorted_item_dict.items():
            print(f"{k}: {v}")
        print()
    else:
        # Else if - user input cannot be found
        print("Please select an appropriate column")

def display_B1(ethnicity, treat_df):
    print(f"\nThe top three treatments for the ethnicity {ethnicity} are \n")
    print(treat_df.head(3).reset_index().to_markdown(tablefmt='github', index=False)) # headers='' to hide headers
    print()

def display_B2(treat_df, ethnicity):
    print(f"Average white blood cell counts for the ethnicity {ethnicity} and treatment type {treat_df.index[0]} \n")
    print(treat_df.reset_index().to_markdown(tablefmt='github', index=False))
    print()

def display_B3(t_df):
    print("Average smoking pack count for \n")
    print(t_df.reset_index().to_markdown(tablefmt='github', index=False))
    print()

# process.py

def unique_columns(df, column_name):
    column_name_list = df[column_name].unique()

    for d in column_name_list:
        print(d)
        d = d.lower()
    return column_name_list

def A1(data):
    patient_ID = input("Enter a patient ID: ").strip()

    found = False
    patient_info = {"Age": "", "Gender": "", "Smoking history": "", "Ethnicity": ""}

    if patient_ID in data[0]:
        found = True
        data = data[(data[0] == patient_ID)]
        #data[(data[0] == patient_ID)][1]
        patient_info = {"Age": data[1], "Gender": data[2], "Smoking history": data[3], "Ethnicity": data[9]}

    display_A1(found, patient_info)

def A2(data):
    # List out all ethnicities
    ethnicity_list = []
    print("List of ethnicities: ")

    for r in data:
        if r[9].lower() not in ethnicity_list:
            ethnicity_list.append(r[9].lower())
            print(r[9])

    ethnicity = input("Enter an ethnicity: ").strip().lower()
    count = input("Enter the amount of display data (all data will be displayed if left blank): ").strip()

    # Else if - user input cannot be found
    if ethnicity not in ethnicity_list:
        ethnicity = "other"

    ethnicity_history = []

    for r in data:
        if(r[9].lower() == ethnicity):
            ethnicity_history.append(r[11])
            ethnicity_history.append(r[12])
            ethnicity_history.append(r[16])
            ethnicity_history.append(r[23])

    if count.isdigit() and len(ethnicity_history) > (int(count) * 4):
        count = int(count) * 4
    else:
        count = len(ethnicity_history)

    display_A2(ethnicity_history, count)

def A3(data):
    # List out all treatment types
    treatment_list = []
    print("List of treatment types: ")

    for r in data:
        if r[7].lower() not in treatment_list:
            treatment_list.append(r[7].lower())
            print(r[7])

    treatment = input("Enter a treatment type: ").strip().lower()
    count = input("Enter the amount of display data (all data will be displayed if left blank): ").strip()

    found = False
    treatment_details = []

    if treatment in treatment_list:
        found = True
        for r in data:
            if(r[7].lower() == treatment and int(r[8]) > 100):
                treatment_details.append(r[1])
                treatment_details.append(r[4])
                treatment_details.append(r[5])
                treatment_details.append(r[6])

    if count.isdigit() and len(treatment_details) > (int(count) * 4):
      count = int(count) * 4
    else:
      count = len(treatment_details)

    display_A3(found, treatment_details, count)

def A4(data, headings):

    columns = {}
    reversed_columns = {}
    index = 0

    for h in headings:
        if h != "Patient_ID" and h != "Tumor_Size_mm":
            columns[h] = index
            reversed_columns[index] = h
            if h == "Blood_Pressure_Pulse":
                break
        index += 1

    print("Please select the column items to display: ")
    index = 0
    for k, v in columns.items():
        index += 1
        print(f"[{index}] {k}")

    found = False

    selected_column = input().strip()
    if selected_column.isdigit():
        selected_column = int(selected_column)

        if selected_column > 3:
            selected_column += 1

        if selected_column in reversed_columns.keys():
            found = True

            print(f"Selected column: {reversed_columns[selected_column]}\n")

            item_dict = {}

            for r in data:
                if reversed_columns[selected_column] != "Stage":
                    key = reversed_columns[selected_column] + " " + r[selected_column]
                else:
                    key = r[selected_column]

                if key not in item_dict.keys():
                    item_dict[key] = 1
                else:
                    value = item_dict[key]
                    item_dict[key] = value + 1

            display_A4(found, item_dict)

def B1(df):
    # List out all ethnicities
    print("List of ethnicities: ")
    ethnicity_list = unique_columns(df, 'Ethnicity')

    ethnicity = input("Enter an ethnicity: ").strip().lower()

    # Else if - user input cannot be found
    if ethnicity not in ethnicity_list:
        ethnicity = "other"

    filtered_df = df[(df['Ethnicity'].str.lower() == ethnicity) & (df['Survival_Months'] > 100)]
    treat_df = filtered_df.groupby('Treatment').size().sort_values(inplace =False, ascending=False)
    treat_df = treat_df.rename('Occurences')

    display_B1(ethnicity, treat_df)

def B2(df):
    # List out all ethnicities
    print("List of ethnicities: ")
    ethnicity_list = unique_columns(df, 'Ethnicity')

    ethnicity = input("Enter an ethnicity: ").strip().lower()
    print()

    # Else if - user input cannot be found
    if ethnicity not in ethnicity_list:
        ethnicity = "other"

    # List out all treatment types
    print("List of treatment types: ")
    treatment_list = unique_columns(df, 'Treatment')

    treatment_type = input("Enter a treatment type: ").strip().lower()
    print()

    filtered_df = df[(df['Ethnicity'].str.lower() == ethnicity) & (df['Treatment'].str.lower() == treatment_type)]

    treat_df = round(filtered_df.groupby('Treatment').White_Blood_Cell_Count.mean(), 2)

    display_B2(treat_df, ethnicity)

def B3(df):
    # List out all tumor locations
    print("List of tumor locations: ")
    unique_columns(df, 'Tumor_Location')

    tumor_location = input("Enter a tumor location: ").strip().lower()
    print()

    filtered_df = df[(df['Blood_Pressure_Pulse'] > 90) & (df['Tumor_Size_mm'] < 15.0)]

    # List out all treatment types
    print("List of treatment types: ")
    treatment_list = unique_columns(df, 'Treatment')

    treatment_type = input("Enter a treatment type: ").strip().lower()
    print()

    treat_df = filtered_df[(filtered_df['Tumor_Location'].str.lower() == tumor_location) & (filtered_df['Treatment'].str.lower() == treatment_type)]
    t_df = round(treat_df.groupby(['Treatment', 'Tumor_Location']).Smoking_Pack_Years.mean(), 2)

    display_B3(t_df)



###


'''
import csv
import pandas as pd

file_name = "lung_cancer_data.csv"

data = []

with open(file_name, encoding="UTF-8") as csv_file:
    # Read the CSV file using the csv.reader() function.
    csv_reader = csv.reader(csv_file)
    headings = next(csv_reader)

    # Store the data in a list (do not convert it to a DataFrame).
    for row in csv_reader:
        data.append(row)

df = pd.read_csv(file_name, encoding='utf-8', index_col='Patient_ID')

B3(df)
'''

###

LINE_WIDTH = 85
def started(msg=""):
    output = f"Operation started: {msg}..."
    dashes = "-" * LINE_WIDTH
    print(f"{dashes}\n{output}\n")

def completed():
    dashes = "-" * LINE_WIDTH
    print(f"\nOperation completed.\n{dashes}\n")

'''
def menu():
    print(f"""Please select one of the following options:
    {"[years]":<10} List unique years
    {"[tally]":<10} Tally up medals
    {"[team]":<10} Tally up medals for each team
    {"[exit]":<10} Exit the program
    """)
    selection = input("Your selection: ")
    return selection.strip().lower()
    '''

def error(msg):
    print(f"Error! {msg}\n")

def display_medal_tally(tally):
    print(f"| {'Gold':<10} | {tally['Gold']:<10} |")
    print(f"| {'Silver':<10} | {tally['Silver']:<10} |")
    print(f"| {'Bronze':<10} | {tally['Bronze']:<10} |")

def display_team_medal_tally(team_tally):
    for team, tally in team_tally.items():
      print(team)
    print(f"\tGold:{tally['Gold']}, Silver:{tally['Silver']}, Bronze:{tally['Bronze']}")

def display_years(years):
    sorted_years = sorted(years, reverse=True)
    for year in sorted_years:
      print(year)

List of tumor locations: 
Lower Lobe
Middle Lobe
Upper Lobe
Enter a tumor location: lower lobe

List of treatment types: 
Surgery
Radiation Therapy
Chemotherapy
Targeted Therapy
Enter a treatment type: surgery

Average smoking pack count for 

| Treatment   | Tumor_Location   |   Smoking_Pack_Years |
|-------------|------------------|----------------------|
| Surgery     | Lower Lobe       |                51.67 |
